# Linear codes over $F_2$

Everything we'll do is really over $F_2$, and the problem of correcting errors, quantum or otherwise will be reduced to bits (but keep in mind that ECC for Analogue is different, also not covered here).

The underlying set of elements of $F_2$ is the set $\{0,1\}$ with addition being bitwise XOR, and multiplication being bitwise AND.

Excersize: Verify that the above definitions meet the axioms of a field. The axioms for multiplication are:

$\forall \alpha 1\cdot \alpha = \alpha \cdot 1 = \alpha$
$\forall \alpha, \beta \in F: \alpha \cdot \beta \in F$ \
$\forall \alpha, \beta \in F: \beta \cdot \alpha = \alpha \cdot \beta$ \
$\forall \alpha, \beta, \gamma \in F: \alpha\cdot(\beta \cdot \gamma) = (\alpha\cdot\beta) \cdot \gamma$ \
$\forall \alpha, \beta, \gamma \in F: \alpha\cdot(\beta \oplus \gamma) = (\alpha\cdot\beta) \oplus (\alpha \cdot \gamma)$ \
$\forall \alpha, \beta, \gamma \in F: \alpha\cdot(\beta \oplus \gamma) = (\alpha\cdot\beta) \oplus (\alpha \cdot \gamma)$ \
$\forall \alpha \neq 0 \exists \beta : \alpha \cdot \beta = 1$ 

Excersize: Verify that bitwise XOR and addition modulu 2 will give you the same answer for adding any number of 1's.

The first concept to understand (everything else builds on it) is bitwise XOR.

Given a vector of $n$ bits $\vec{(b_0 ..b_{n-1})}$, we form a new vector $\vec{(c,b_0 ..b_{n-1})}$ with $c= \oplus_{i=0..n-1}  b_i$.

Excersize: Check that $(\oplus_{i=0..n-1} b_i) \oplus c_0 = 0$

You can now check that if any odd number of bits from the $b_i$'s of $c$ are flipped, the result of $(\oplus_{i=1..n-1} b_i) \oplus c$ is 1.

The equation:
$$(\oplus_{i=1..n-1} b_i) \oplus c = 0$$
Is sometimes reffered to a single parity checksum (SPC). 

What's happening here in terms of vector spaces ?

We started with a vector $\vec{(b_0 ..b_{n-1})}$ in $F_2^n$, and ended up with a vector $\vec{(c,b_0 ..b_{n-1})}$ in $F_2^{n+1}$. The set of all vectors of the form $\vec{((\oplus_{i=0..n-1} b_i) ,b_0 ..b_{n-1})}$ is a vector space on it's own, with dimension $n$.

So we might as well have started with $F_2^{n+1}$, and restriced ourselves to to the subspace of vectors that satisfy: $\vec{c_0, ..c_{n}}$ such that $\oplus_{i=0..n} c_i = 0$. 

Excersize: Verify that $\vec{((\oplus_{i=0..n-1} b_i) ,b_0 ..b_{n-1})}$  with the usual coordinate-wise addition is a vector space over $F_2$. Check that it is isomorphic to $F_2^n$, and so it is an $n$ dimensional subspace of $F_2^{n+1}$.

The idea is that we can define these SPC's over different sets of indices from $\{0..n-1\}$, which we can denote by $I \subset \{0..n-1\}$ and obtain many linear equations, say $m$ of them:

$$ ( \oplus_{i\in I_0} b_i)  = 0 $$
$$ ( \oplus_{i\in I_1} b_i)  = 0 $$
.
.
.
$$ ( \oplus_{i\in I_m} b_i)  = 0 $$

Excersize: Define a polynomial $c(x)=c \oplus b_0 \cdot x \oplus b_1 \cdot x^2 \oplus ... \oplus b_{n-1}x^{n}$. Verify that $c(1) = 0$.


Excersize: Define a polynomial $b(x)=b_0 \oplus b_1 \cdot x \oplus ... \oplus b_{n-1}x^{n-1}$. Verify that $x\cdot b(x)$ evaluated at $x=1$ equals $c_0$. 

Excersize: Prove that $b(x)$ and $c(x)$ as defined above, satisfy $x \cdot b(x) mod (x+1) = c(x)$

